<a href="https://colab.research.google.com/github/Naitik6803/ML-Models/blob/master/classification_with_Fashion_MNIST_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Cloth Classification deep learning using MNIST datset**

In [ ]:
!pip install -U tensorflow_datasets


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import math
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import tqdm.auto
tqdm.tqdm=tqdm.auto.tqdm



In [ ]:
dataset, meta_data=tfds.load('fashion_mnist',as_supervised=True,with_info=True)
train_data,test_data= dataset['train'],dataset['test']
num_train_examples = meta_data.splits['train'].num_examples
num_test_examples = meta_data.splits['test'].num_examples

In [ ]:
class_names = meta_data.features['label'].names
# print("Class names: {}".format(class_names))
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

train_data =  train_data.map(normalize)
test_data  =  test_data.map(normalize)


train_data =  train_data.cache()
test_data  =  test_data.cache()

In [ ]:
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(train_data.take(25)):
    image = image.numpy().reshape((28,28))
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image, cmap=plt.cm.binary)
    plt.xlabel(class_names[label])
plt.show()

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
BATCH_SIZE = 32
train_data = train_data.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_data = test_data.cache().batch(BATCH_SIZE)

In [ ]:
model.fit(train_data, epochs=6, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data, steps=math.ceil(num_test_examples/BATCH_SIZE))
print('Loss on test dataset :',test_loss)
print('Accuracy on test dataset:', test_accuracy)

In [ ]:
for test_images, test_labels in test_data.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

In [ ]:
predictions.shape

In [ ]:
predictions[0]

In [ ]:
a=np.argmax(predictions[0])
print(a)
print(test_labels[0])

## Plotting to showcase in proper way

In [ ]:
def plot_image(i, predictions_array, true_labels, images):
  predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img[...,0], cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'green'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1]) 
  predicted_label = np.argmax(predictions_array)
  
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('green')

In [ ]:
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)